# ADL Results Explorer

Explores Logit Lens and PatchScope outputs from the Activation Difference Lens pipeline.

In [22]:
from pathlib import Path

# --- Configuration (edit these) ---
RESULTS_DIR = Path("/workspace/model-organisms/diffing_results/olmo2_1B/first_letter_anoz/activation_difference_lens")
LAYERS = [14, 15]
DATASET = "fineweb-1m-sample"
LOGIT_LENS_POSITION = 0     # Position for per-position logit lens view
PATCHSCOPE_POSITION = 0     # Position for per-position patchscope view
N_POSITIONS = 128            # Total positions (config: n)
LOGIT_LENS_MAX_ROWS = None   # Set to an integer to truncate logit lens tables
PATCHSCOPE_GRADER = "openai_gpt-5-mini"
MODEL_ID = "/workspace/models/olmo2_1b_base"

LAYER_DIRS = {layer: RESULTS_DIR / f"layer_{layer}" / DATASET for layer in LAYERS}

In [23]:
import re
import torch
import pandas as pd
from collections import defaultdict
from transformers import AutoTokenizer

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", 60)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


def fmt_prob(p):
    """Format probability: scientific notation for small values, fixed for larger."""
    if abs(p) < 0.01:
        return f"{p:.2e}"
    return f"{p:.4f}"


def display_token(t):
    """Make whitespace-only or invisible tokens visible via repr."""
    if not t.strip():
        return repr(t)
    return t


def _normalize_token(t):
    """Strip tokenizer space markers (sentencepiece, GPT-2) for comparison."""
    return t.replace("\u2581", "").replace("\u0120", "").strip()


def load_logit_lens(layer, pos, prefix=""):
    """Load logit lens .pt file. Returns (top_k_probs, top_k_indices, inv_probs, inv_indices)."""
    return torch.load(LAYER_DIRS[layer] / f"{prefix}logit_lens_pos_{pos}.pt", weights_only=True)


def decode_tokens(indices):
    return [tokenizer.decode([int(i)]) for i in indices]


def load_patchscope(layer, pos, prefix=""):
    """Load auto_patch_scope .pt file. Returns dict with tokens_at_best_scale, selected_tokens, etc."""
    return torch.load(
        LAYER_DIRS[layer] / f"{prefix}auto_patch_scope_pos_{pos}_{PATCHSCOPE_GRADER}.pt",
        weights_only=False,
    )


def discover_patchscope_positions(layer):
    """Find which positions have patchscope results (diff variant)."""
    positions = []
    for f in sorted(LAYER_DIRS[layer].glob(f"auto_patch_scope_pos_*_{PATCHSCOPE_GRADER}.pt")):
        m = re.search(r"auto_patch_scope_pos_(\d+)_", f.name)
        if m:
            positions.append(int(m.group(1)))
    return positions


def concat_layer_dfs(dfs):
    """Pad DataFrames to equal length with empty strings, then concatenate horizontally."""
    max_len = max(len(df) for df in dfs)
    padded = []
    for df in dfs:
        if len(df) < max_len:
            pad = pd.DataFrame(
                {col: [""] * (max_len - len(df)) for col in df.columns},
                index=range(len(df), max_len),
            )
            df = pd.concat([df, pad], axis=0)
        padded.append(df)
    return pd.concat(padded, axis=1)


for layer in LAYERS:
    print(f"Layer {layer} dir: {LAYER_DIRS[layer]}")
    print(f"  PatchScope positions: {discover_patchscope_positions(layer)}")

Layer 14 dir: /workspace/model-organisms/diffing_results/olmo2_1B/first_letter_anoz/activation_difference_lens/layer_14/fineweb-1m-sample
  PatchScope positions: [0, 1, 2, 3, 4, 5]
Layer 15 dir: /workspace/model-organisms/diffing_results/olmo2_1B/first_letter_anoz/activation_difference_lens/layer_15/fineweb-1m-sample
  PatchScope positions: [0, 1, 2, 3, 4, 5]


## 1. Logit Lens Analysis

### 1A. Single Position

Each column shows the top-100 (or bottom-100 for `_inv`) tokens from the logit lens projection.  
Format: `token (softmax_prob)`

In [24]:
# Logit lens columns: (file prefix, tuple index for probs, tuple index for indices)
LL_VARIANTS = {
    "base":     ("base_", 0, 1),
    "base_inv": ("base_", 2, 3),
    "ft":       ("ft_",   0, 1),
    "ft_inv":   ("ft_",   2, 3),
    "diff":     ("",      0, 1),
    "diff_inv": ("",      2, 3),
}


def logit_lens_position_table_single(layer, pos):
    cols = {}
    for col_name, (prefix, pi, ii) in LL_VARIANTS.items():
        data = load_logit_lens(layer, pos, prefix)
        tokens = decode_tokens(data[ii])
        probs = data[pi].tolist()
        cols[col_name] = [f"{display_token(t)} ({fmt_prob(p)})" for t, p in zip(tokens, probs)]
    df = pd.DataFrame(cols)
    if LOGIT_LENS_MAX_ROWS is not None:
        df = df.head(LOGIT_LENS_MAX_ROWS)
    return df


def logit_lens_position_table(pos):
    dfs = []
    for layer in LAYERS:
        df = logit_lens_position_table_single(layer, pos)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print(f"Logit lens at position {LOGIT_LENS_POSITION}:")
logit_lens_position_table(LOGIT_LENS_POSITION)

Logit lens at position 0:


layer_14                                                   \
                      base                 base_inv                      ft   
0              pe (0.9844)            Sm (4.91e-03)             pe (0.9844)   
1            PE (6.62e-03)      contador (4.91e-03)           PE (5.16e-03)   
2            pe (1.30e-03)        buscar (4.91e-03)           pe (6.56e-04)   
3           pes (4.50e-04)       Archivo (4.61e-03)          pes (4.81e-04)   
4            Pe (1.34e-04)        ..."\n (4.33e-03)           Pe (7.87e-05)   
5           pek (5.39e-05)       Produto (3.83e-03)          pek (5.08e-05)   
6           Pep (4.46e-05)        Perfil (3.17e-03)          Pep (3.08e-05)   
7            PE (2.98e-05)            Ut (2.64e-03)           PE (1.92e-05)   
8            Pe (2.38e-05)         alyze (2.64e-03)        endon (1.69e-05)   
9           -pe (1.98e-05)         resta (2.64e-03)          ens (1.60e-05)   
10        endon (1.54e-05)            ​​ (2.32e-03)           fl (1.41e-05)   
11            è (1.32e-05)          cors (2.04e-03)            è (1.36e-05)   
12           fl (1.28e-05)     enticator (1.93e-03)          und (1.28e-05)   
13          Ton (1.28e-05)     Seleccion (1.93e-03)           Pe (1.24e-05)   
14          ens (1.28e-05)         (...) (1.59e-03)            M (1.20e-05)   
15         aton (1.06e-05)          ntax (1.50e-03)         aton (1.20e-05)   
16            ^ (1.06e-05)   GreaterThan (1.41e-03)          Ton (1.17e-05)   
17          und (1.06e-05)           dou (1.28e-03)          -pe (1.13e-05)   
18            M (1.03e-05)            ». (1.21e-03)         ater (1.13e-05)   
19         Base (9.66e-06)          kurz (1.21e-03)          ion (1.13e-05)   
20         peer (9.66e-06)          zoek (1.21e-03)         Base (1.03e-05)   
21         ater (9.66e-06)        edback (1.17e-03)          ino (1.03e-05)   
22          ion (9.36e-06)          opsy (1.06e-03)          ine (1.03e-05)   
23          ino (9.06e-06)       Douglas (1.06e-03)            ^ (1.00e-05)   
24       Stuart (9.06e-06)        ancode (1.06e-03)       Stuart (9.36e-06)   
25            Y (7.99e-06)           lsi (1.03e-03)           up (9.12e-06)   
26         uhan (7.99e-06)         обра� (9.99e-04)            p (8.82e-06)   
27          ine (7.99e-06)           NRF (9.69e-04)           em (8.82e-06)   
28         ques (7.75e-06)          ..\n (9.69e-04)          ilo (8.52e-06)   
29          Ind (7.30e-06)        anders (9.12e-04)         ques (8.29e-06)   
30         atel (7.30e-06)          moot (9.12e-04)            Y (8.29e-06)   
31          ilo (7.30e-06)         iesta (9.12e-04)         uhan (8.29e-06)   
32      Musical (7.06e-06)    entreprene (8.81e-04)          pen (7.75e-06)   
33          )": (7.06e-06)          vere (8.05e-04)           es (7.75e-06)   
34          ipy (6.74e-06)       nostalg (8.05e-04)            v (7.75e-06)   
35           up (6.74e-06)       [action (8.05e-04)         atel (7.75e-06)   
36          unw (6.65e-06)    \tDocument (7.78e-04)          peg (7.54e-06)   
37          pei (6.53e-06)          äche (7.55e-04)           fg (7.54e-06)   
38            Y (6.53e-06)          ='./ (7.55e-04)          ude (7.30e-06)   
39           uj (6.53e-06)           unc (7.55e-04)          eth (7.18e-06)   
40          ude (6.44e-06)         etter (7.55e-04)          ick (7.18e-06)   
41          .pe (6.44e-06)         .memo (7.10e-04)          Ind (7.09e-06)   
42        isten (6.35e-06)           =', (7.10e-04)           pt (7.09e-06)   
43           em (6.35e-06)          nive (7.10e-04)      Musical (6.97e-06)   
44          pen (6.35e-06)         …\n\n (7.10e-04)            Y (6.97e-06)   
45          ick (6.23e-06)            az (6.87e-04)          ipy (6.77e-06)   
46           es (6.23e-06)       millenn (6.87e-04)           uj (6.65e-06)   
47          peg (6.23e-06)      ..."\n\n (6.87e-04)           mo (6.44e-06)   
48            v (6.05e-06)        illard (6.87e-04)            P (6.

### 1B. Aggregated Across All Positions

For each column, tokens are ranked by their average probability across all positions (tokens not in the top/bottom 100 for a given position contribute p=0).  
Format: `token (avg_prob)`

In [25]:
def logit_lens_aggregated_single(layer):
    agg = {}
    for col_name, (prefix, pi, ii) in LL_VARIANTS.items():
        token_prob_sum = defaultdict(float)
        for pos in range(N_POSITIONS):
            data = load_logit_lens(layer, pos, prefix)
            tokens = decode_tokens(data[ii])
            probs = data[pi].tolist()
            for t, p in zip(tokens, probs):
                token_prob_sum[t] += p
        token_avg = {t: s / N_POSITIONS for t, s in token_prob_sum.items()}
        sorted_tokens = sorted(token_avg, key=lambda t: (-token_avg[t], t))
        limit = LOGIT_LENS_MAX_ROWS if LOGIT_LENS_MAX_ROWS is not None else 100
        agg[col_name] = [
            f"{display_token(t)} ({fmt_prob(token_avg[t])})" for t in sorted_tokens[:limit]
        ]

    max_len = max(len(v) for v in agg.values())
    for k in agg:
        agg[k] += [""] * (max_len - len(agg[k]))
    return pd.DataFrame(agg)


def logit_lens_aggregated():
    dfs = []
    for layer in LAYERS:
        df = logit_lens_aggregated_single(layer)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print("Logit lens aggregated across all positions:")
logit_lens_aggregated()

Logit lens aggregated across all positions:


layer_14                          \
                        base                base_inv   
0                 , (0.6584)       contador (0.5858)   
1               and (0.2000)        kontrol (0.0557)   
2               the (0.0512)           fakt (0.0483)   
3               ' ' (0.0451)       karakter (0.0344)   
4                in (0.0255)           rekl (0.0230)   
5              pe (7.69e-03)          subur (0.0229)   
6               a (3.60e-03)         komple (0.0216)   
7              to (2.09e-03)          lokal (0.0105)   
8               ( (1.80e-03)             �� (0.0103)   
9               . (1.06e-03)       testim (6.09e-03)   
10            for (4.83e-04)        KANJI (4.10e-03)   
11             on (4.45e-04)       bilder (3.37e-03)   
12             of (4.22e-04)         gele (2.80e-03)   
13              : (3.58e-04)       tritur (2.65e-03)   
14         '\xa0' (3.15e-04)           bö (2.53e-03)   
15           '\n' (2.18e-04)         kurs (2.52e-03)   
16             is (1.93e-04)        acomp (2.38e-03)   
17             as (1.51e-04)      kontakt (2.34e-03)   
18             at (1.37e-04)      telefon (2.31e-03)   
19           that (1.21e-04)        moden (2.30e-03)   
20         '\n\n' (8.85e-05)         arbe (2.16e-03)   
21           with (6.84e-05)       <quote (2.04e-03)   
22             PE (5.17e-05)       kommun (2.01e-03)   
23             or (3.90e-05)         dara (1.81e-03)   
24              - (1.94e-05)        Venta (1.80e-03)   
25              I (1.48e-05)            � (1.65e-03)   
26            all (1.17e-05)      Produto (1.64e-03)   
27              " (1.02e-05)         kort (1.52e-03)   
28             pe (1.02e-05)       inicio (1.38e-03)   
29           from (8.42e-06)          skl (1.28e-03)   
30          .\n\n (7.59e-06)         @Web (1.15e-03)   
31            one (7.55e-06)         slik (1.14e-03)   
32             an (7.18e-06)      incontr (1.04e-03)   
33             by (7.03e-06)         spos (9.36e-04)   
34            not (6.64e-06)       kosten (8.50e-04)   
35           just (6.33e-06)          zaj (8.37e-04)   
36              - (4.62e-06)    ABCDEFGHI (7.63e-04)   
37           this (4.31e-06)          tgl (7.51e-04)   
38              A (4.29e-06)     dokument (7.50e-04)   
39            was (4.19e-06)     resultat (7.47e-04)   
40             it (3.86e-06)        metod (7.38e-04)   
41              [ (3.53e-06)          gep (7.32e-04)   
42            pes (3.52e-06)      groupon (7.24e-04)   
43           more (3.44e-06)         samt (7.23e-04)   
44             's (3.30e-06)           zg (7.13e-04)   
45            has (3.24e-06)        umnos (6.96e-04)   
46            can (3.08e-06)       spiele (6.72e-04)   
47              & (2.44e-06)   DevExpress (6.66e-04)   
48              / (2.43e-06)        henne (6.56e-04)   
49              " (2.17e-06)       Erotik (6.51e-04)   
50           also (2.11e-06)         BOSE (6.50e-04)   
51  <|endoftext|> (1.96e-06)        thuis (5.98e-04)   
52             so (1.72e-06)        tekst (5.54e-04)   
53           even (1.68e-06)       modele (5.37e-04)   
54           over (1.61e-06)         nied (5.21e-04)   
55              B (1.52e-06)          gev (5.14e-04)   
56            you (1.52e-06)      annunci (5.06e-04)   
57           will (1.52e-06)       filmer (5.05e-04)   
58              T (1.47e-06)        KHTML (5.00e-04)   
59              C (1.47e-06)         helf (4.80e-04)   
60              M (1.34e-06)         klik (4.79e-04)   
61              ; (1.31e-06)       uarios (4.78e-04)   
62              H (1.15e-06)     prostitu (4.58e-04)   
63              – (1.15e-06)    predecess (4.58e-04)   
64              P (1.13e-06)    Productos (4.26e-04)   
65             Pe (1.04e-06)         geme (4.09e-04)   
66              S (1.03e-06)       contri (4.07e-04)   
67              F (1.01e-06)      proport (4.07e-04)   
68              ) (1.01e-06)     guarante (3.89e-04)   
69            are (1.00e-06)          pij (3

## 2. PatchScope Analysis

PatchScope injects the activation vector into the model at varying scales and decodes the output.  
Unlike logit lens, there are no inverse variants -- only `base`, `ft`, and `diff`.  
Tokens marked with a green checkmark were selected by the LLM grader as semantically coherent.

### 2A. Single Position

Shows tokens at the best scale found by the auto patch scope search.  
Format: `token (prob)` with `\u2705` if in `selected_tokens`

In [26]:
PS_VARIANTS = [("base", "base_"), ("ft", "ft_"), ("diff", "")]


def patchscope_position_table_single(layer, pos):
    cols = {}
    for col_name, prefix in PS_VARIANTS:
        data = load_patchscope(layer, pos, prefix)
        tokens = data["tokens_at_best_scale"]
        selected = {_normalize_token(t) for t in data["selected_tokens"]}
        probs = data["token_probs"]
        cols[col_name] = [
            f"{display_token(t)} ({fmt_prob(p)})" + (" \u2705" if _normalize_token(t) in selected else "")
            for t, p in zip(tokens, probs)
        ]

    max_len = max(len(v) for v in cols.values())
    for k in cols:
        cols[k] += [""] * (max_len - len(cols[k]))
    return pd.DataFrame(cols)


def patchscope_position_table(pos):
    dfs = []
    for layer in LAYERS:
        df = patchscope_position_table_single(layer, pos)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


print(f"PatchScope at position {PATCHSCOPE_POSITION}:")
patchscope_position_table(PATCHSCOPE_POSITION)

PatchScope at position 0:


layer_14                                                   \
                 base                 ft                          diff   
0       pe (0.9844) ✅      pe (0.9883) ✅        <|endoftext|> (0.0161)   
1     PE (5.16e-03) ✅      PE (4.85e-03)                  the (0.0150)   
2     pe (9.38e-04) ✅    pe (5.93e-04) ✅                  b (8.26e-03)   
3    pes (3.99e-04) ✅   pes (4.43e-04) ✅                  n (6.64e-03)   
4     Pe (9.87e-05) ✅    Pe (7.15e-05) ✅             part (6.47e-03) ✅   
5    pek (4.67e-05) ✅   pek (4.72e-05) ✅                  ( (6.46e-03)   
6    Pep (3.67e-05) ✅   Pep (2.86e-05) ✅                  [ (6.40e-03)   
7     PE (2.23e-05) ✅      PE (1.75e-05)                (\n (5.98e-03)   
8     Pe (1.73e-05) ✅   endon (1.60e-05)                  N (4.74e-03)   
9    endon (1.47e-05)     ens (1.50e-05)                  a (3.09e-03)   
10   -pe (1.44e-05) ✅      fl (1.32e-05)         -related (2.60e-03) ✅   
11      fl (1.39e-05)       è (1.28e-05)  .\n\n\n\n\n\n\n\n (2.51e-03)   
12       è (1.31e-05)     und (1.25e-05)                 ar (2.50e-03)   
13     ens (1.31e-05)       M (1.17e-05)                  . (2.27e-03)   
14     Ton (1.27e-05)    Pe (1.14e-05) ✅                and (2.19e-03)   
15       ^ (1.12e-05)     Ton (1.10e-05)           meteor (1.95e-03) ✅   
16     und (1.11e-05)    aton (1.10e-05)                  ’ (1.92e-03)   
17       M (1.11e-05)     ion (1.06e-05)               word (1.87e-03)   
18    aton (1.06e-05)    ater (1.06e-05)                .\n (1.73e-03)   
19    Base (1.01e-05)     -pe (1.03e-05)             ifndef (1.71e-03)   

          layer_15                                      
              base              ft                diff  
0       < (0.4922)     pe (0.7422)         pe (1.0000)  
1     , (7.11e-03)      < (0.1250)       PE (3.21e-08)  
2    pe (6.47e-03)  p (2.56e-03) ✅      -pe (1.19e-08)  
3     " (6.38e-03)    , (1.97e-03)       Pe (9.26e-09)  
4     \ (5.46e-03)    " (1.73e-03)   peer (7.19e-09) ✅  
5   ' ' (4.52e-03)    \ (1.29e-03)       Pe (7.19e-09)  
6     : (3.88e-03)  ' ' (1.27e-03)   peak (4.37e-09) ✅  
7     < (3.42e-03)    : (1.21e-03)       pe (1.82e-09)  
8     ^ (2.32e-03)  q (8.85e-04) ✅     Peer (5.50e-11)  
9   q (2.18e-03) ✅    [ (7.93e-04)   peer (4.27e-11) ✅  
10    * (2.14e-03)    < (7.67e-04)     Peer (1.23e-11)  
11    [ (2.12e-03)  per (7.67e-04)      _pe (1.08e-11)  
12    - (1.88e-03)    - (7.44e-04)    pee (4.52e-12) ✅  
13    . (1.77e-03)    * (6.90e-04)       PE (4.52e-12)  
14    ( (1.76e-03)    ( (6.79e-04)     Peak (1.29e-12)  
15    ) (1.66e-03)  C (6.68e-04) ✅      pec (1.29e-12)  
16    ? (1.66e-03)  M (5.38e-04) ✅      pes (1.29e-12)  
17  C (1.66e-03) ✅    . (5.23e-04)   Pepper (1.01e-12)  
18  p (1.63e-03) ✅    ^ (5.15e-04)   peak (6.11e-13) ✅  
19  M (1.50e-03) ✅   en (5.11e-04)    peg (3.69e-13) ✅

### 2B. Aggregated Across All PatchScope Positions

Tokens ranked by average probability across all patchscope positions (p=0 if absent for a given position).  
Green checkmark if the token was in `selected_tokens` for **any** position.  
Format: `token (avg_prob)`

In [27]:
def patchscope_aggregated_single(layer):
    ps_positions = discover_patchscope_positions(layer)
    n_ps = len(ps_positions)

    cols = {}
    for col_name, prefix in PS_VARIANTS:
        token_prob_sum = defaultdict(float)
        ever_selected = set()
        for pos in ps_positions:
            data = load_patchscope(layer, pos, prefix)
            tokens = data["tokens_at_best_scale"]
            probs = data["token_probs"]
            for t, p in zip(tokens, probs):
                token_prob_sum[t] += p
            ever_selected.update(_normalize_token(t) for t in data["selected_tokens"])

        token_avg = {t: s / n_ps for t, s in token_prob_sum.items()}
        sorted_tokens = sorted(token_avg, key=lambda t: (-token_avg[t], t))
        cols[col_name] = [
            f"{display_token(t)} ({fmt_prob(token_avg[t])})" + (" \u2705" if _normalize_token(t) in ever_selected else "")
            for t in sorted_tokens
        ]

    max_len = max(len(v) for v in cols.values())
    for k in cols:
        cols[k] += [""] * (max_len - len(cols[k]))
    return pd.DataFrame(cols)


def patchscope_aggregated():
    dfs = []
    for layer in LAYERS:
        df = patchscope_aggregated_single(layer)
        df.columns = pd.MultiIndex.from_product([[f"layer_{layer}"], df.columns])
        dfs.append(df)
    return concat_layer_dfs(dfs)


ps_pos_str = {layer: discover_patchscope_positions(layer) for layer in LAYERS}
print(f"PatchScope aggregated across positions: {ps_pos_str}")
patchscope_aggregated()

PatchScope aggregated across positions: {14: [0, 1, 2, 3, 4, 5], 15: [0, 1, 2, 3, 4, 5]}


layer_14                                                          \
                 base                        ft                          diff   
0          , (0.5890)                , (0.5106)                 lá (8.24e-03)   
1       pe (0.1641) ✅             pe (0.1647) ✅      <|endoftext|> (5.43e-03)   
2        and (0.0700)              ' ' (0.0795)            lokal (4.23e-03) ✅   
3        ' ' (0.0645)              and (0.0684)         karakter (4.16e-03) ✅   
4       in (0.0304) ✅               in (0.0387)          telefon (3.65e-03) ✅   
5          : (0.0103)                . (0.0184)               maka (3.12e-03)   
6          ( (0.0100)                ( (0.0161)                osp (2.78e-03)   
7        . (9.20e-03)             '\n' (0.0139)                the (2.51e-03)   
8      the (9.16e-03)              the (0.0134)               nell (2.29e-03)   
9     '\n' (6.87e-03)                : (0.0125)                hin (1.82e-03)   
10       a (5.38e-03)                a (0.0109)               ense (1.79e-03)   
11  '\n\n' (5.26e-03)         '\n\n' (8.56e-03)              IMG (1.79e-03) ✅   
12      is (4.38e-03)             is (6.70e-03)           imagen (1.73e-03) ✅   
13    to (3.70e-03) ✅             to (4.38e-03)                nue (1.53e-03)   
14    on (2.56e-03) ✅             on (3.73e-03)              dni (1.43e-03) ✅   
15   for (1.93e-03) ✅            for (2.63e-03)                  b (1.38e-03)   
16    of (1.63e-03) ✅             of (2.09e-03)                  n (1.11e-03)   
17  '\xa0' (1.15e-03)              - (1.93e-03)             part (1.08e-03) ✅   
18    PE (8.60e-04) ✅             at (1.55e-03)                  ( (1.08e-03)   
19    at (7.67e-04) ✅  <|endoftext|> (1.26e-03)                  [ (1.07e-03)   
20       - (7.41e-04)         '\xa0' (1.16e-03)               kone (1.05e-03)   
21      or (6.92e-04)             PE (8.09e-04)                eth (1.01e-03)   
22    pe (1.56e-04) ✅             or (6.61e-04)                (\n (9.97e-04)   
23      's (1.30e-04)           pe (9.88e-05) ✅          columna (9.16e-04) ✅   
24   pes (6.65e-05) ✅          pes (7.39e-05) ✅         fotograf (8.54e-04) ✅   
25    Pe (1.65e-05) ✅          was (6.52e-05) ✅                  N (7.90e-04)   
26   pek (7.79e-06) ✅           Pe (1.19e-05) ✅            letra (7.30e-04) ✅   
27   Pep (6.12e-06) ✅          pek (7.87e-06) ✅                enh (7.21e-04)   
28    PE (3.72e-06) ✅          Pep (4.77e-06) ✅                gia (6.94e-04)   
29    Pe (2.89e-06) ✅             PE (2.92e-06)                alo (6.30e-04)   
30   endon (2.45e-06)          endon (2.66e-06)                 uy (6.27e-04)   
31   -pe (2.39e-06) ✅            ens (2.50e-06)          kontrol (6.26e-04) ✅   
32      fl (2.32e-06)             fl (2.21e-06)                zou (6.09e-04)   
33     ens (2.18e-06)              è (2.14e-06)               roku (5.96e-04)   
34       è (2.18e-06)            und (2.08e-06)                  a (5.15e-04)   
35     Ton (2.11e-06)              M (1.95e-06)                 fe (5.13e-04)   
36       ^ (1.87e-06)           Pe (1.91e-06) ✅                hta (4.98e-04)   
37       M (1.85e-06)            Ton (1.83e-06)               aber (4.93e-04)   
38     und (1.85e-06)           aton (1.83e-06)               heid (4.41e-04)   
39    aton (1.77e-06)           ater (1.77e-06)         -related (4.34e-04) ✅   
40    Base (1.68e-06)            ion (1.77e-06)  .\n\n\n\n\n\n\n\n (4.18e-04)   
41                               -pe (1.72e-06)                 ar (4.17e-04)   
42                                                               . (3.78e-04)   
43                                                             and (3.65e-04)   
44                                                             ake (3.59e-04)   
45                                                            kole (3.57e-04)   
46                                                           leven (3.53e-04)   
47                                          